In [245]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pvlib
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.pvsystem import PVSystem
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

In [190]:
kerman_d=pd.read_csv("./testds/DailyKerman.csv",skiprows=13)
kerman_h=pd.read_csv("./testds/HourlyKerman.csv",skiprows=13)
kerman_loc=Location(longitude=57.0788,latitude=30.2830,tz="Iran")

In [191]:
def select_day(df,year,month,day):
    return df[(df["year"]==year)&(df["month"]==month)&(df["day"]==day)]

In [192]:
kerman_d

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,WS10M,T2M
0,2024,1,1,2.94,2.93,1.53,2.75,7.07
1,2024,1,2,3.36,4.44,1.35,1.52,6.47
2,2024,1,3,3.26,3.08,1.83,4.23,6.94
3,2024,1,4,2.84,2.80,1.66,3.53,5.64
4,2024,1,5,4.02,6.90,0.87,3.00,6.11
...,...,...,...,...,...,...,...,...
362,2024,12,28,3.66,5.02,1.27,4.37,6.75
363,2024,12,29,3.68,5.28,1.18,5.07,8.37
364,2024,12,30,2.42,0.47,1.96,4.37,6.82
365,2024,12,31,1.81,0.38,1.48,4.54,5.61


In [193]:
kerman_h

,YEAR,MO,DY,HR,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,WS10M,T2M
0,2024,1,1,0,0.0,0.0,0.0,2.92,3.38
1,2024,1,1,1,0.0,0.0,0.0,3.07,2.95
2,2024,1,1,2,0.0,0.0,0.0,3.18,2.54
3,2024,1,1,3,0.0,0.0,0.0,3.30,2.07
4,2024,1,1,4,0.0,0.0,0.0,3.41,1.67
...,...,...,...,...,...,...,...,...,...
8803,2025,1,1,19,0.0,0.0,0.0,2.06,2.00
8804,2025,1,1,20,0.0,0.0,0.0,2.42,1.18
8805,2025,1,1,21,0.0,0.0,0.0,2.83,0.31
8806,2025,1,1,22,0.0,0.0,0.0,3.27,-0.68


# Daily Dataset

In [194]:
# select_day(kerman_h,2024,1,1)
kerman_d = kerman_d.rename(columns={'YEAR': 'year', 'MO': 'month', 'DY': 'day'}).copy()
kerman_d["date"]=pd.to_datetime(kerman_d[["year","month","day"]])
kerman_d['date_iran'] = kerman_d['date'].dt.tz_localize('Asia/Tehran',nonexistent='shift_forward')#.apply(lambda ts: ts.replace(hour=8,minute=30))
kerman_d

,year,month,day,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,WS10M,T2M,date,date_iran
0,2024,1,1,2.94,2.93,1.53,2.75,7.07,2024-01-01,2024-01-01 00:00:00+03:30
1,2024,1,2,3.36,4.44,1.35,1.52,6.47,2024-01-02,2024-01-02 00:00:00+03:30
2,2024,1,3,3.26,3.08,1.83,4.23,6.94,2024-01-03,2024-01-03 00:00:00+03:30
3,2024,1,4,2.84,2.80,1.66,3.53,5.64,2024-01-04,2024-01-04 00:00:00+03:30
4,2024,1,5,4.02,6.90,0.87,3.00,6.11,2024-01-05,2024-01-05 00:00:00+03:30
...,...,...,...,...,...,...,...,...,...,...
362,2024,12,28,3.66,5.02,1.27,4.37,6.75,2024-12-28,2024-12-28 00:00:00+03:30
363,2024,12,29,3.68,5.28,1.18,5.07,8.37,2024-12-29,2024-12-29 00:00:00+03:30
364,2024,12,30,2.42,0.47,1.96,4.37,6.82,2024-12-30,2024-12-30 00:00:00+03:30
365,2024,12,31,1.81,0.38,1.48,4.54,5.61,2024-12-31,2024-12-31 00:00:00+03:30


# Hourly Dataset

In [195]:
kerman_h = kerman_h.rename(columns={'YEAR': 'year', 'MO': 'month', 'DY': 'day',"HR":"hour"}).copy()
kerman_h["date"]=pd.to_datetime(kerman_h[["year","month","day","hour"]])
kerman_h['date_iran'] = kerman_h['date'].dt.tz_localize('Asia/Tehran',nonexistent='shift_forward')
kerman_h

,year,month,day,hour,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,WS10M,T2M,date,date_iran
0,2024,1,1,0,0.0,0.0,0.0,2.92,3.38,2024-01-01 00:00:00,2024-01-01 00:00:00+03:30
1,2024,1,1,1,0.0,0.0,0.0,3.07,2.95,2024-01-01 01:00:00,2024-01-01 01:00:00+03:30
2,2024,1,1,2,0.0,0.0,0.0,3.18,2.54,2024-01-01 02:00:00,2024-01-01 02:00:00+03:30
3,2024,1,1,3,0.0,0.0,0.0,3.30,2.07,2024-01-01 03:00:00,2024-01-01 03:00:00+03:30
4,2024,1,1,4,0.0,0.0,0.0,3.41,1.67,2024-01-01 04:00:00,2024-01-01 04:00:00+03:30
...,...,...,...,...,...,...,...,...,...,...,...
8803,2025,1,1,19,0.0,0.0,0.0,2.06,2.00,2025-01-01 19:00:00,2025-01-01 19:00:00+03:30
8804,2025,1,1,20,0.0,0.0,0.0,2.42,1.18,2025-01-01 20:00:00,2025-01-01 20:00:00+03:30
8805,2025,1,1,21,0.0,0.0,0.0,2.83,0.31,2025-01-01 21:00:00,2025-01-01 21:00:00+03:30
8806,2025,1,1,22,0.0,0.0,0.0,3.27,-0.68,2025-01-01 22:00:00,2025-01-01 22:00:00+03:30


# Model Params

In [196]:
selected_pannel="aSiMicro03036"
modual_infos=pd.read_csv("./simulation prams/module_params_custom_material.csv",index_col="Model")
temp_infos =pd.read_csv("./simulation prams/Temperature_Parameters_for_Each_PV_Model.csv",index_col="Model",usecols=["Model","a","b","deltaT"])
cec_inverter = pvlib.pvsystem.retrieve_sam(name="CECInverter") 

inverter_params =cec_inverter["ABB__PVI_3_0_OUTD_S_US__208V_"]
module_params = modual_infos.loc[selected_pannel]
temp_params=temp_infos.loc[selected_pannel]

In [197]:
kerman_d

,year,month,day,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,WS10M,T2M,date,date_iran
0,2024,1,1,2.94,2.93,1.53,2.75,7.07,2024-01-01,2024-01-01 00:00:00+03:30
1,2024,1,2,3.36,4.44,1.35,1.52,6.47,2024-01-02,2024-01-02 00:00:00+03:30
2,2024,1,3,3.26,3.08,1.83,4.23,6.94,2024-01-03,2024-01-03 00:00:00+03:30
3,2024,1,4,2.84,2.80,1.66,3.53,5.64,2024-01-04,2024-01-04 00:00:00+03:30
4,2024,1,5,4.02,6.90,0.87,3.00,6.11,2024-01-05,2024-01-05 00:00:00+03:30
...,...,...,...,...,...,...,...,...,...,...
362,2024,12,28,3.66,5.02,1.27,4.37,6.75,2024-12-28,2024-12-28 00:00:00+03:30
363,2024,12,29,3.68,5.28,1.18,5.07,8.37,2024-12-29,2024-12-29 00:00:00+03:30
364,2024,12,30,2.42,0.47,1.96,4.37,6.82,2024-12-30,2024-12-30 00:00:00+03:30
365,2024,12,31,1.81,0.38,1.48,4.54,5.61,2024-12-31,2024-12-31 00:00:00+03:30


# Tests

## Input Daily

In [228]:
input_d=kerman_d[["date_iran","ALLSKY_SFC_SW_DWN","ALLSKY_SFC_SW_DNI","ALLSKY_SFC_SW_DIFF","T2M","WS10M"]].copy()
input_d.set_index("date_iran",drop=True,inplace=True)
# ['linear', 'time', 'index', 'values', 'nearest', 'zero', 'slinear', 'quadratic', 
#  'cubic', 'barycentric', 'krogh', 'spline', 'polynomial', 'from_derivatives', 'piecewise_polynomial', 'pchip', 'akima', 'cubicspline']
# input_d = input_d.resample('h').interpolate(method="from_derivatives")
input_d=input_d.rename(columns={"ALLSKY_SFC_SW_DWN":"ghi",
                          "ALLSKY_SFC_SW_DNI":"dni",
                          "ALLSKY_SFC_SW_DIFF":"dhi",
                          "T2M":"temp_air",
                          "WS10M":"wind_speed"})
input_d["ghi"]=input_d["ghi"]* 1000.0 
input_d["dni"]=input_d["dni"]* 1000.0 
input_d["dhi"]=input_d["dhi"]* 1000.0 
input_d

,ghi,dni,dhi,temp_air,wind_speed
date_iran,,,,,
2024-01-01 00:00:00+03:30,2940.0,2930.0,1530.0,7.07,2.75
2024-01-02 00:00:00+03:30,3360.0,4440.0,1350.0,6.47,1.52
2024-01-03 00:00:00+03:30,3260.0,3080.0,1830.0,6.94,4.23
2024-01-04 00:00:00+03:30,2840.0,2800.0,1660.0,5.64,3.53
2024-01-05 00:00:00+03:30,4020.0,6900.0,870.0,6.11,3.00
...,...,...,...,...,...
2024-12-28 00:00:00+03:30,3660.0,5020.0,1270.0,6.75,4.37
2024-12-29 00:00:00+03:30,3680.0,5280.0,1180.0,8.37,5.07
2024-12-30 00:00:00+03:30,2420.0,470.0,1960.0,6.82,4.37


## Input Hourly

In [229]:
input_h=kerman_h[["date_iran","ALLSKY_SFC_SW_DWN","ALLSKY_SFC_SW_DNI","ALLSKY_SFC_SW_DIFF","T2M","WS10M"]].copy()
input_h.set_index("date_iran",drop=True,inplace=True)

input_h=input_h.rename(columns={"ALLSKY_SFC_SW_DWN":"ghi",
                          "ALLSKY_SFC_SW_DNI":"dni",
                          "ALLSKY_SFC_SW_DIFF":"dhi",
                          "T2M":"temp_air",
                          "WS10M":"wind_speed"})
input_h["ghi"]=input_h["ghi"]
input_h["dni"]=input_h["dni"]
input_h["dhi"]=input_h["dhi"]
input_h

,ghi,dni,dhi,temp_air,wind_speed
date_iran,,,,,
2024-01-01 00:00:00+03:30,0.0,0.0,0.0,3.38,2.92
2024-01-01 01:00:00+03:30,0.0,0.0,0.0,2.95,3.07
2024-01-01 02:00:00+03:30,0.0,0.0,0.0,2.54,3.18
2024-01-01 03:00:00+03:30,0.0,0.0,0.0,2.07,3.30
2024-01-01 04:00:00+03:30,0.0,0.0,0.0,1.67,3.41
...,...,...,...,...,...
2025-01-01 19:00:00+03:30,0.0,0.0,0.0,2.00,2.06
2025-01-01 20:00:00+03:30,0.0,0.0,0.0,1.18,2.42
2025-01-01 21:00:00+03:30,0.0,0.0,0.0,0.31,2.83


In [230]:
delta_h = input_h.index.to_series().diff().dt.total_seconds().div(3600)

In [231]:
module_params=dict(module_params)
module_params["gamma_pdc"]=-0.0038
pdc0=module_params["Parallel_Strings"]*module_params["Isco"]*module_params["Voco"]
module_params["pdc0"]=700
module_params=pd.Series(module_params)

In [232]:
inverter_params=dict(inverter_params)
inverter_params["pdc0"]=inverter_params["Pdco"]
inverter_params=pd.Series(inverter_params)

## Modeling

In [236]:
module_params = {
    'pdc0':      700, 
    'gamma_pdc': -0.0038,
}
inverter_params = {
    'pdc0': 5000,
    'eta_inv_nom': 0.96
}

In [243]:
ac = pvlib.pvwatts.pvwatts_ac(
    effective_irradiance  = input_d["ghi"],       # your daily GHI series [W/m²]
    temp_cell    = input_d['temp_air'],  # °C
    wind_speed  = input_d['wind_speed']
)

AttributeError: module 'pvlib' has no attribute 'pvwatts'

In [240]:
sandia_modules = pvlib.pvsystem.retrieve_sam(name="SandiaMod")
cec_inverter = pvlib.pvsystem.retrieve_sam(name="CECInverter") 
temp_params=TEMPERATURE_MODEL_PARAMETERS["sapm"]["open_rack_glass_glass"] 
module = sandia_modules["Canadian_Solar_CS5P_220M___2009_"]
inverter = cec_inverter["ABB__PVI_3_0_OUTD_S_US__208V_"]

system = PVSystem(
    surface_tilt=30,
    surface_azimuth=180,
    module_parameters=module_params,
    inverter_parameters=inverter_params,
    # temperature_model_parameters=temp_params,
    # modules_per_string=1, # increase the number of moduals in EACH string  so total number of moduals = modules_per_string * strings_per_inverter
    # strings_per_inverter=1 #increase the number of stings connected to one inverter . this means the number of prallel strings 
    
)
model_chain = ModelChain.with_pvwatts(system,kerman_loc)

ValueError: Could not infer temperature model from ModelChain.system.  If Arrays are used to construct the PVSystem, check that all Arrays in ModelChain.system.arrays have parameters for the same temperature model. If Arrays are not used, check that the PVSystem attributes `racking_model` and `module_type` are valid.

## D Predictions

In [238]:
model_chain.run_model(input_d)

ModelChain: 
  name: None
  clearsky_model: ineichen
  transposition_model: perez
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: pvwatts_dc
  ac_model: pvwatts_inverter
  aoi_model: physical_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: pvwatts_losses

In [239]:
d_res=pd.DataFrame()
d_ac=model_chain.results.ac.values
d_res["Date"]=input_d.index
d_res["P"]=d_ac
d_res.set_index("Date")

,P
Date,
2024-01-01 00:00:00+03:30,1.512563
2024-01-02 00:00:00+03:30,5.961620
2024-01-03 00:00:00+03:30,4.886848
2024-01-04 00:00:00+03:30,0.625626
2024-01-05 00:00:00+03:30,12.930301
...,...
2024-12-28 00:00:00+03:30,9.094414
2024-12-29 00:00:00+03:30,9.087720
2024-12-30 00:00:00+03:30,0.000000


## H Predictions

In [112]:
model_chain.run_model(input_h)

ModelChain: 
  name: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: sandia_inverter
  aoi_model: sapm_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses

In [113]:
h_res=pd.DataFrame()
h_ac=model_chain.results.ac.values
h_res["Date"]=input_h.index
h_res["P"]=h_ac
h_res.set_index("Date").resample("D").sum()

,P
Date,
2024-01-01 00:00:00+03:30,302.917124
2024-01-02 00:00:00+03:30,395.585462
2024-01-03 00:00:00+03:30,334.356858
2024-01-04 00:00:00+03:30,286.843549
2024-01-05 00:00:00+03:30,544.209083
...,...
2024-12-28 00:00:00+03:30,455.172102
2024-12-29 00:00:00+03:30,466.450711
2024-12-30 00:00:00+03:30,123.227720
